<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 4: West Nile Virus Prediction

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from pycaret.classification import *

In [2]:
train = pd.read_csv('../datasets/train_clean.csv')
test= pd.read_csv('../datasets/test_clean.csv')

In [3]:
train.head()

,latitude,longitude,wnvpresent,week,month_5,month_6,month_7,month_8,month_9,month_10,species_culex pipiens,species_culex pipiens/restuans,species_culex restuans,species_others,tmax,tmin,tavg,dewpoint,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,relativehumidity
0,41.867108,-87.654224,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249
1,41.867108,-87.654224,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249
2,41.862292,-87.648860,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249
3,41.896282,-87.655232,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249
4,41.907645,-87.760886,0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.0,43.5,56.5,40.5,0.09,29.41,30.11,9.5,3.5,10.2,54.921249


In [4]:
test.head()

,id,latitude,longitude,week,month_5,month_6,month_7,month_8,month_9,month_10,species_culex pipiens,species_culex pipiens/restuans,species_culex restuans,species_others,tmax,tmin,tavg,dewpoint,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,relativehumidity
0,1,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192
1,2,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192
2,3,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192
3,4,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192
4,5,41.95469,-87.800991,24,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,85.5,58.5,72.0,52.5,0.005,29.185,29.88,1.8,10.5,5.25,50.256192


## Baseline Model

## Pycaret

### Setting Up Environment

First we use the `setup()` function to initialize the environment in pycaret and create the transformation pipeline to prepare the data for modeling and deployment. It takes two mandatory parameters: our train dataset and target column (our y variable) which is `'wnvpresent'`. 

When `setup()` is executed, PyCaret's inference algorithm will automatically infer the data types for all features based on certain properties and display a table containing the features and their inferred data types after setup() is executed.

Our train and test datasets have gone through `OneHotEncoding` for the categorical features in the 1st notebook. Pycaret is able to scale our data using the `normalize=True` parameter and we will be using z-score which is the same as `StandardScaler` (or Z-score normalization). Continuous features like `'relativehumidity'` has a wide range and due to differences in magnitude of features, our model may neglect units of higher number. 

Pycaret is able to detect and classify the features of the dataset. However, the features that `OneHotEncoding` had transformed was detected to be of categorical in nature instead. To bypass this, we had to list out these features to override the classification before running the dataset through Pycaret environment.


In [5]:
pce_1 = setup(data = train, target = 'wnvpresent', session_id = 5, 
              data_split_stratify=True, 
              normalize=True, normalize_method='zscore', #to scale our data
              numeric_features = ['week', 'month_5', 'month_6','month_7','month_8','month_9','month_10',
                                  'species_culex pipiens','species_culex pipiens/restuans','species_culex restuans','species_others'
                                 ] #since the categorical features has been converted, it ought to be treated as numerical features
             )

,Description,Value
0,session_id,5
1,Target,wnvpresent
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(8475, 25)"
5,Missing Values,False
6,Numeric Features,24
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [6]:
%time compare_models(n_select=2, sort = 'AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9472,0.8438,0.0719,0.6700,0.1225,0.1131,0.1909,0.2220
lightgbm,Light Gradient Boosting Machine,0.9447,0.8406,0.1438,0.4641,0.2149,0.1949,0.2332,0.0500
ada,Ada Boost Classifier,0.9461,0.8328,0.0344,0.3817,0.0607,0.0547,0.0976,0.0830
xgboost,Extreme Gradient Boosting,0.9390,0.8301,0.1656,0.3484,0.2204,0.1939,0.2098,0.3160
lr,Logistic Regression,0.9461,0.8038,0.0000,0.0000,0.0000,0.0000,0.0000,0.6270
lda,Linear Discriminant Analysis,0.9461,0.7971,0.0000,0.0000,0.0000,0.0000,0.0000,0.0130
rf,Random Forest Classifier,0.9396,0.7890,0.0969,0.2719,0.1371,0.1162,0.1338,0.1640
nb,Naive Bayes,0.2787,0.7816,0.9906,0.0690,0.1291,0.0314,0.1235,0.0080
et,Extra Trees Classifier,0.9331,0.7504,0.1250,0.2445,0.1620,0.1324,0.1413,0.1450
knn,K Neighbors Classifier,0.9408,0.7061,0.0469,0.2679,0.0773,0.0617,0.0889,0.0800


CPU times: user 5.63 s, sys: 500 ms, total: 6.13 s
Wall time: 21.4 s


[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=5, subsample=1.0, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False),
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=-1, num_leaves=31, 

Once the environment was setup, we were able to generate models based on default parameters in the Pycaret library. The `compare_models` function trains all models in the model library and scores them using stratified cross validation for metric evaluation. The output prints a score grid that shows average Accuracy, AUC, Recall, Precision, F1, Kappa, and MCC across the folds (10 by default) along with training times. The score grid with the highest performing metrics are highlighted accordingly. The grid by default is sorted using 'AUC' (highest to lowest) as the Kaggle competition will be looking at ROC-AUC scores for the submission. Therefore, we will be looking to optimize AUC as much as possible, while maintaining a good score in Accuracy and Precision.

The `compare_models` also returned the best performing models based on AUC scores with the `n_select=2` parameter which gives us the top 2 highest performing models which are Gradient Boosting Classifier and Light Gradient Boosting Machine Classifier. After narrowing down to these two models, we will attempt to increase its performance by tuning its hyperparameters. Pycaret also has the function to optimize the models generated by using the `tune_model` that helps tune the hyperparameters of the models. As mentioned earlier, we will be focusing on AUC score and the function will focus in optimizing based on AUC score by passing in the `optimize='AUC'` parameter.

### Light Gradient Boosting Model

In [7]:
#create lgbm model based on default param
lgbm_model = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9495,0.8453,0.2500,0.5714,0.3478,0.3257,0.3562
1,0.9495,0.8608,0.1562,0.6250,0.2500,0.2335,0.2956
2,0.9427,0.8241,0.1250,0.4000,0.1905,0.1691,0.2006
3,0.9477,0.8515,0.1250,0.5714,0.2051,0.1894,0.2503
4,0.9477,0.8815,0.2812,0.5294,0.3673,0.3427,0.3615
5,0.9494,0.8744,0.1562,0.6250,0.2500,0.2335,0.2955
6,0.9444,0.8170,0.0625,0.4000,0.1081,0.0949,0.1412
7,0.9376,0.8250,0.0938,0.2727,0.1395,0.1151,0.1331
8,0.9477,0.8133,0.1562,0.5556,0.2439,0.2256,0.2756


In [8]:
#test scores based on default param
pred_lgbm = predict_model(lgbm_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9410,0.8642,0.1241,0.3617,0.1848,0.1617,0.1871


In [9]:
#tuning lgbm model
%time tuned_lgbm = tune_model(lgbm_model, n_iter = 50, optimize='AUC', tuner_verbose=False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9478,0.8623,0.2188,0.5385,0.3111,0.2890,0.3211
1,0.9512,0.8797,0.1250,0.8000,0.2162,0.2046,0.3045
2,0.9477,0.8471,0.0938,0.6000,0.1622,0.1498,0.2229
3,0.9460,0.8649,0.0938,0.5000,0.1579,0.1433,0.1996
4,0.9410,0.8592,0.2188,0.4118,0.2857,0.2579,0.2721
5,0.9511,0.8823,0.1250,0.8000,0.2162,0.2046,0.3045
6,0.9477,0.8347,0.0312,1.0000,0.0606,0.0575,0.1721
7,0.9376,0.8260,0.0938,0.2727,0.1395,0.1151,0.1331
8,0.9494,0.8357,0.0938,0.7500,0.1667,0.1566,0.2539


CPU times: user 3.74 s, sys: 216 ms, total: 3.96 s
Wall time: 21 s


In [10]:
#showing the best params
print(tuned_lgbm)

LGBMClassifier(bagging_fraction=0.7, bagging_freq=4, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.6,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.9,
               n_estimators=240, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5, reg_alpha=0.01, reg_lambda=1e-06, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [11]:
#generate interface that will display various plots
evaluate_model(tuned_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [12]:
#test scores based on tuned model
pred_holdout_lgbm = predict_model(tuned_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9426,0.8617,0.0949,0.3714,0.1512,0.1321,0.1662


We analyze the performance of the Light Generated Boosting Machine (LGBM) model by using the `evaluate_model()` function which displays a user interface for all of the available plots for a given model. It internally uses the plot_model() function. The main focus in this portion is to evaluate the model based on the ROC-AUC score and graph. With an AUC score of 0.86, it suggests the 'west nile virus' and 'no west nile virus' populations are well separated, and the model is nearly as good as it can get. It also shows that the model is better at predicting 'no west nile virus' populations at at lower decision threshold, with the ROC for class 0 being higher and steeper up to 0.2 False Positive Rate.

The scores generated after running the model through the holdout (or 'test' set after Pycaret internal train-test split) also showed good predictive power with Accuracy and AUC being 0.94 and 0.86 respectively. From the markdown table below, we see the improvement in the model after the hyperparameter tuning as well:

| Model (parameter) | Train Accuracy Score | Test Accuracy Score | Train AUC | Test AUC |
|---|---|---|---|---|
| Baseline (default)  | - | - | - | - |
| LGBM (default) | 0.9447 | 0.9410 | 0.8406 | 0.8642 |
| LGBM  (tuned) | 0.9459 | 0.9426 | 0.8617 | 0.8642 |

### Gradient Boosting Model

In [13]:
#create gbc model based on default params
%time gbc_model = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9529,0.8509,0.2188,0.7000,0.3333,0.3158,0.3745
1,0.9495,0.8764,0.0625,1.0000,0.1176,0.1120,0.2436
2,0.9477,0.8561,0.0312,1.0000,0.0606,0.0575,0.1721
3,0.9477,0.8485,0.0312,1.0000,0.0606,0.0575,0.1721
4,0.9528,0.8612,0.1562,0.8333,0.2632,0.2504,0.3487
5,0.9460,0.8576,0.0312,0.5000,0.0588,0.0528,0.1148
6,0.9477,0.8444,0.0625,0.6667,0.1143,0.1060,0.1934
7,0.9427,0.8286,0.0625,0.3333,0.1053,0.0898,0.1250
8,0.9460,0.8278,0.0312,0.5000,0.0588,0.0528,0.1148


CPU times: user 1.12 s, sys: 18.2 ms, total: 1.14 s
Wall time: 3.31 s


In [14]:
#test scores based on default param
pred_gbc = predict_model(gbc_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.9477,0.8494,0.0584,0.6667,0.1074,0.0996,0.1869


In [15]:
%time tuned_gbc = tune_model(gbc_model, n_iter = 50, optimize='AUC', tuner_verbose=False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9478,0.8559,0.1875,0.5455,0.2791,0.2586,0.2991
1,0.9478,0.8622,0.0938,0.6000,0.1622,0.1498,0.2229
2,0.9477,0.8534,0.0625,0.6667,0.1143,0.1060,0.1934
3,0.9477,0.8597,0.0312,1.0000,0.0606,0.0575,0.1721
4,0.9477,0.8643,0.2188,0.5385,0.3111,0.2889,0.3210
5,0.9494,0.8730,0.0625,1.0000,0.1176,0.1120,0.2436
6,0.9477,0.8409,0.0312,1.0000,0.0606,0.0575,0.1721
7,0.9410,0.8318,0.0938,0.3333,0.1463,0.1256,0.1535
8,0.9444,0.8348,0.0938,0.4286,0.1538,0.1371,0.1812


CPU times: user 2.48 s, sys: 176 ms, total: 2.65 s
Wall time: 1min 21s


In [16]:
#showing the best params
print(tuned_gbc)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=2,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0.0005,
                           min_impurity_split=None, min_samples_leaf=2,
                           min_samples_split=10, min_weight_fraction_leaf=0.0,
                           n_estimators=270, n_iter_no_change=None,
                           presort='deprecated', random_state=5, subsample=1.0,
                           tol=0.0001, validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [17]:
#generate interface that will display various plots
evaluate_model(tuned_gbc)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [18]:
#test scores based on tuned model
pred_holdout_gbc = predict_model(tuned_gbc)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.9457,0.8706,0.0803,0.4783,0.1375,0.1239,0.1796


We did the same steps as when generating the model for LGBM for Gradient Boosting Classifier (GBC). After optimizing the model, the scores showed an improvement as well. From the ROC-AUC graph, we can see it has a similar score to LGBM but it does show an improvement by 0.01 with AUC score of 0.87. Again, this model also shows that it is better at predicting 'no west nile virus' populations at at lower decision threshold, with the ROC for class 0 being higher and steeper up to 0.2 False Positive Rate.

The scores generated after running the model through the holdout set also showed good predictive power with Accuracy and AUC being 0.94 and 0.87 respectively. Below, we are able to see the difference in scores before hyperparameter tuning and evaluate which models showed the best potential:

| Model (parameter) | Train Accuracy Score | Test Accuracy Score | Train AUC | Test AUC |
|---|---|---|---|---|
| Baseline (default)  | - | - | - | - |
| LGBM (default) | 0.9447 | 0.9410 | 0.8406 | 0.8642 |
| LGBM  (tuned) | 0.9459 | 0.9426 | 0.8617 | 0.8642 |
| GBC (default) | 0.9477 | 0.9477 | 0.8438 | 0.8494 |
| GBC (tuned) | 0.9454 | 0.9457 | 0.8490 | 0.8706 |

Both LGBM and GBC outperformed out baseline models and after comparing the two models, it seems like GBC has the best scores when predicting the holdout set in Pycaret, even though LGBM had slighly better scores in the training. Thus, we will be using Gradient Boosting Classifier as our final model to predict the test dataset and submit it to Kaggle.


In [19]:
#finalizing the model by training it on the complete dataset before it is deployed for predicting the test datasest
final_gbc = finalize_model(tuned_gbc)

In [20]:
#predicting on test dataset
pred_unseen_gbc = predict_model(final_gbc, data = test)

## Kaggle Submission

In [21]:
#making function to change the score and making appropriate dataframe for Kaggle submission
def kaggle_submission(preds, model_name):
    
    #changing the scores
    preds['Prob'] = preds['Score']
    preds.loc[preds['Label'] == 0, 'Prob'] = 1-preds['Prob']
    
    #making appropriate dataframe for submission
    submission = pd.merge(test['id'], preds['Prob'], left_index = True, right_index = True)
    
    submission.rename({'id' : 'Id',
                      'Prob' : 'WnvPresent'},
                     inplace = True,
                     axis = 1)
    
    #export
    submission.to_csv(f'../datasets/submission_{model_name}.csv', index=False)
    
    return submission

In [22]:
kaggle_submission(pred_unseen_gbc, 'gbc')

,Id,WnvPresent
0,1,0.0090
1,2,0.0040
2,3,0.0069
3,4,0.0002
4,5,0.0002
...,...,...
116288,116289,0.0001
116289,116290,0.0001
116290,116291,0.0001
116291,116292,0.0001


## Cost-Benefit Analysis of Spraying

## Conclusions and Recommendation